In [27]:
import pandas as pd

# Load datasets
data_10sample = pd.read_csv('C:/ml/Data/sin_noisy_10sample.csv')
data_20sample = pd.read_csv('C:/ml/Data/sin_noisy_20sample.csv')
data_40sample = pd.read_csv('C:/ml/Data/sin_noisy_40sample.csv')
data_80sample = pd.read_csv('C:/ml/Data/sin_noisy_80sample.csv')

# Combine datasets into a dictionary
data_dict = {
    10: data_10sample,
    20: data_20sample,
    40: data_40sample,
    80: data_80sample
}




In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

degrees = [1, 3, 8]

results = []

for n_samples, data in data_dict.items():
    X = data[['x']].values
    y = data['noisy_y'].values
    
    for degree in degrees:
        poly = PolynomialFeatures(degree)
        X_poly = poly.fit_transform(X)
        
        model = LinearRegression()
        model.fit(X_poly, y)
        
        # การทำนายโดยใช้ training set
        y_pred_train = model.predict(X_poly)
        rmse_train = np.sqrt(mean_squared_error(y, y_pred_train))
        
        # การทำนายโดยใช้ cross-validation
        scores = cross_val_score(model, X_poly, y, scoring='neg_mean_squared_error', cv=5)
        rmse_cv = np.sqrt(-scores.mean())
        
        # บันทึกผลลัพธ์
        results.append({
            'n_samples': n_samples,
            'degree': degree,
            'rmse_train': rmse_train,
            'rmse_cv': rmse_cv
        })

results_df = pd.DataFrame(results)


In [29]:
# แสดงผลลัพธ์
print(results_df)

# แสดง coefficients สำหรับ polynomial degree ต่าง ๆ
coefficients = {}

for n_samples, data in data_dict.items():
    X = data[['x']].values
    y = data['noisy_y'].values
    
    for degree in degrees:
        poly = PolynomialFeatures(degree)
        X_poly = poly.fit_transform(X)
        
        model = LinearRegression()
        model.fit(X_poly, y)
        
        coefficients[(n_samples, degree)] = model.coef_

# แสดง coefficients
for key, value in coefficients.items():
    print(f'n_samples: {key[0]}, degree: {key[1]}')
    print(value)

    n_samples  degree  rmse_train    rmse_cv
0          10       1    0.572799   1.059066
1          10       3    0.226547   0.593959
2          10       8    0.060663  38.949689
3          20       1    0.431594   0.600687
4          20       3    0.271959   0.610174
5          20       8    0.193577  55.724494
6          40       1    0.544472   0.753276
7          40       3    0.292903   0.369609
8          40       8    0.271811  34.116675
9          80       1    0.499048   0.671636
10         80       3    0.273691   0.362947
11         80       8    0.257766  15.174893
n_samples: 10, degree: 1
[0.         1.07222121]
n_samples: 10, degree: 3
[ 0.          2.77721639  0.52324301 -2.59796523]
n_samples: 10, degree: 8
[  0.           1.01530933  -5.10343645  20.00527424  31.28377247
 -67.33168913 -67.42238562  55.62458056  50.54873512]
n_samples: 20, degree: 1
[0.         1.05476692]
n_samples: 20, degree: 3
[ 0.          2.33255904  0.09990481 -2.09900934]
n_samples: 20, degree: